<a href="https://colab.research.google.com/github/DEHailey/Kaggle/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EA%B3%BC%EC%A0%9C_%EA%B9%80%ED%98%84%EA%B2%BD_0531_3_50th_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 문제 정의
- 통신사의 고객 유지를 위해 고객 이탈을 예측하는 머신러닝 모델 개발하기
- 고객의 개인정보및 가입정보를 입력으로 하여 이탈 여부를 분류하는 이진 분류 문제
    - 평가지표: AUC
- 베이스라인 코드를 개선하여 모델의 예측 성능을 높여 주세요!!
    - 베이스라인 코드실행시 Public Score(AUC): 0.82182




- 통신사 고객 이탈 여부 데이터셋 다운로드 받아 구글드라이브에 올리기
    - 학습 데이터: https://drive.google.com/file/d/1dF_pTpMLbLNJ7gcN0f7LqR-qyze2kLFR/view?usp=sharing
    - 테스트 데이터: https://drive.google.com/file/d/1aUEQvsRul_5VnmPMqYrQVu-eqI2Fl3Z8/view?usp=sharing
    - 제출 파일 양식: https://drive.google.com/file/d/1d7An7Ab07df62bsEjChkdz5t_MzuEocF/view?usp=sharing

- 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수


In [ ]:
DATA_PATH = "/content/drive/MyDrive/머신러닝/data/"
DATA_PATH

'/content/drive/MyDrive/머신러닝/data/'

- 시드값

In [ ]:
SEED = 42

- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}customer_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}customer_test.csv") # 테스트 데이터
train.shape , test.shape

((4930, 21), (2113, 20))

# 통신사 고객 이탈 여부 데이터셋 설명
- 이탈 여부 컬럼이 타겟으로 1이 이탈!
- 성별 컬럼
    - 1 = 여성, 0 = 남성
- 여부 관련 컬럼
    - 1 = yes, 0 = no
- 약정기간 컬럼
    - 0 = 약정 없음
    - 1 = 1년 약정, 2 = 2년 약정

In [ ]:
train.head()

,ID,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷이용방식,인터넷보안서비스사용여부,...,기기방화벽서비스사용여부,인터넷기술지원서비스사용여부,스트리밍TV여부,스트리밍영화여부,약정기간,지로여부,지불방법,한달요금,총지불요금,이탈여부
0,train_0,1,0,0,0,23,1,0,광,0,...,0,1,0,0,0,0,메일,79.10,1783.75,0
1,train_1,1,0,0,0,1,1,0,디지털,0,...,0,0,0,0,0,0,메일,45.40,45.40,1
2,train_2,0,0,0,0,23,1,1,광,0,...,1,1,1,1,0,0,메일,104.05,2470.10,1
3,train_3,1,0,0,0,11,1,0,디지털,0,...,1,1,0,1,1,1,신용카드,64.90,697.25,0
4,train_4,1,0,0,0,5,1,1,광,0,...,0,0,1,0,0,0,전자,85.20,474.80,1


# 데이터 분석(Data analysis)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4930 entries, 0 to 4929
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4930 non-null   object 
 1   성별              4930 non-null   int64  
 2   고령자여부           4930 non-null   int64  
 3   기혼여부            4930 non-null   int64  
 4   부양가족여부          4930 non-null   int64  
 5   가입기간            4930 non-null   int64  
 6   집전화이용여부         4930 non-null   int64  
 7   다중회선여부          4930 non-null   int64  
 8   인터넷이용방식         3859 non-null   object 
 9   인터넷보안서비스사용여부    4930 non-null   int64  
 10  인터넷백업서비스사용여부    4930 non-null   int64  
 11  기기방화벽서비스사용여부    4930 non-null   int64  
 12  인터넷기술지원서비스사용여부  4930 non-null   int64  
 13  스트리밍TV여부        4930 non-null   int64  
 14  스트리밍영화여부        4930 non-null   int64  
 15  약정기간            4930 non-null   int64  
 16  지로여부            4930 non-null   int64  
 17  지불방법            4930 non-null   o

- 결측치 처리하기

In [ ]:
train.isnull().sum()

ID                   0
성별                   0
고령자여부                0
기혼여부                 0
부양가족여부               0
가입기간                 0
집전화이용여부              0
다중회선여부               0
인터넷이용방식           1071
인터넷보안서비스사용여부         0
인터넷백업서비스사용여부         0
기기방화벽서비스사용여부         0
인터넷기술지원서비스사용여부       0
스트리밍TV여부             0
스트리밍영화여부             0
약정기간                 0
지로여부                 0
지불방법                 0
한달요금                 0
총지불요금                0
이탈여부                 0
dtype: int64

In [ ]:
test.isnull().sum()

ID                  0
성별                  0
고령자여부               0
기혼여부                0
부양가족여부              0
가입기간                0
집전화이용여부             0
다중회선여부              0
인터넷이용방식           455
인터넷보안서비스사용여부        0
인터넷백업서비스사용여부        0
기기방화벽서비스사용여부        0
인터넷기술지원서비스사용여부      0
스트리밍TV여부            0
스트리밍영화여부            0
약정기간                0
지로여부                0
지불방법                0
한달요금                0
총지불요금               0
dtype: int64

In [ ]:
internet_type = train["인터넷이용방식"].mode()[0]

In [ ]:
train["인터넷이용방식"] = train["인터넷이용방식"].fillna(internet_type)
test["인터넷이용방식"] = test["인터넷이용방식"].fillna(internet_type)

In [ ]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

In [ ]:
train.describe()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,인터넷기술지원서비스사용여부,스트리밍TV여부,스트리밍영화여부,약정기간,지로여부,한달요금,총지불요금,이탈여부
count,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000,4930.000000
mean,0.497972,0.160446,0.485396,0.298783,32.807505,0.904868,0.421907,0.288844,0.344219,0.347465,0.295132,0.386207,0.388641,0.701826,0.412373,64.894787,2309.829148,0.265314
std,0.500047,0.367057,0.499837,0.457771,24.684877,0.293427,0.493914,0.453271,0.475161,0.476213,0.456148,0.486928,0.487491,0.838039,0.492312,30.074560,2282.572526,0.441545
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.675000,411.337500,0.000000
50%,0.000000,0.000000,0.000000,0.000000,29.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,70.450000,1412.150000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,56.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,89.850000,3845.412500,1.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,118.650000,8684.800000,1.000000


In [ ]:
# 이상치 처리
train['총지불요금'] = train['총지불요금'].where((train['지불방법'] != '메일') | (train['총지불요금'] <= 3112), 1412.15)

In [ ]:
# 이상치 처리
train['총지불요금'] = train['총지불요금'].where((train['지불방법'] != '전자') | (train['총지불요금'] <= 7862.25), 8574.7)

In [ ]:
# 새로운 특성_1
train['가입기간_약정기간_비율'] = train['가입기간'] / (train['약정기간'] + 1)
test['가입기간_약정기간_비율'] = test['가입기간'] / (test['약정기간'] + 1)

In [ ]:
# 새로운 특성_2
train['스TV_스영화'] = train['스트리밍TV여부'] + train['스트리밍영화여부']
test['스TV_스영화'] = test['스트리밍TV여부'] + test['스트리밍영화여부']

In [ ]:
# 새로운 특성_3
train['부가서비스_5종'] = train['인터넷보안서비스사용여부'] + train['인터넷백업서비스사용여부'] + train['기기방화벽서비스사용여부'] + train['인터넷기술지원서비스사용여부'] + train['다중회선여부']
test['부가서비스_5종'] = test['인터넷보안서비스사용여부'] + test['인터넷백업서비스사용여부'] + test['기기방화벽서비스사용여부'] + test['인터넷기술지원서비스사용여부'] + test['다중회선여부']

In [ ]:
# 새로운 특성_4
train['총지불요금_편차'] = train.groupby('가입기간')['총지불요금'].transform(lambda x: np.std(x))
test = pd.merge(test, train.groupby('가입기간')['총지불요금'].std().reset_index(name='총지불요금_편차'), on='가입기간', how='left').fillna(0)

In [ ]:
# 새로운 특성_5
train['전자_가중치'] = (train['지불방법'] == '전자').astype(int)
test['전자_가중치'] = (test['지불방법'] == '전자').astype(int)

In [ ]:
train.columns

Index(['ID', '성별', '고령자여부', '기혼여부', '부양가족여부', '가입기간', '집전화이용여부', '다중회선여부',
       '인터넷이용방식', '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부',
       '인터넷기술지원서비스사용여부', '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법',
       '한달요금', '총지불요금', '이탈여부', '가입기간_약정기간_비율', '스TV_스영화', '부가서비스_5종',
       '총지불요금_편차', '전자_가중치'],
      dtype='object')

In [ ]:
# 특성으로 사용할 변수 추가하기
cols = ['성별', '고령자여부', '기혼여부', '부양가족여부', '가입기간', '집전화이용여부', '다중회선여부',
       '인터넷이용방식', '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부',
       '인터넷기술지원서비스사용여부', '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법',
       '한달요금', '총지불요금', '가입기간_약정기간_비율', '스TV_스영화', '부가서비스_5종',
       '총지불요금_편차', '전자_가중치']
train_ft = train[cols].copy()
test_ft = test[cols].copy() # 테스트 데이터
train_ft.shape, test_ft.shape

((4930, 24), (2113, 24))

In [ ]:
family_cols = ['고령자여부', '기혼여부','부양가족여부']

In [ ]:
# 학습데이터
from itertools import combinations
for col in combinations(family_cols, 2):
   train_ft[f"{col[0]}_{col[1]}"] = train_ft[col[0]].astype(str) + "_" + train_ft[col[1]].astype(str)

In [ ]:
# 테스트데이터
from itertools import combinations
for col in combinations(family_cols, 2):
   test_ft[f"{col[0]}_{col[1]}"] = test_ft[col[0]].astype(str) + "_" + test_ft[col[1]].astype(str)

In [ ]:
service_cols = ['집전화이용여부','다중회선여부','인터넷이용방식','인터넷보안서비스사용여부','인터넷백업서비스사용여부','기기방화벽서비스사용여부']

In [ ]:
# 학습데이터
from itertools import combinations
for col in combinations(service_cols, 2):
   train_ft[f"{col[0]}_{col[1]}"] = train_ft[col[0]].astype(str) + "_" + train_ft[col[1]].astype(str)

In [ ]:
# 테스트데이터
from itertools import combinations
for col in combinations(service_cols, 2):
   test_ft[f"{col[0]}_{col[1]}"] = test_ft[col[0]].astype(str) + "_" + test_ft[col[1]].astype(str)

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import category_encoders as ce

In [ ]:
train_ft.nunique()

성별                              2
고령자여부                           2
기혼여부                            2
부양가족여부                          2
가입기간                           73
집전화이용여부                         2
다중회선여부                          2
인터넷이용방식                         2
인터넷보안서비스사용여부                    2
인터넷백업서비스사용여부                    2
기기방화벽서비스사용여부                    2
인터넷기술지원서비스사용여부                  2
스트리밍TV여부                        2
스트리밍영화여부                        2
약정기간                            3
지로여부                            2
지불방법                            4
한달요금                         1432
총지불요금                        4530
가입기간_약정기간_비율                  157
스TV_스영화                         3
부가서비스_5종                        6
총지불요금_편차                       73
전자_가중치                          2
고령자여부_기혼여부                      4
고령자여부_부양가족여부                    4
기혼여부_부양가족여부                     4
집전화이용여부_다중회선여부                  3
집전화이용여부_인터넷이용방식                 3
집전화이용여부_인터넷보안서

In [ ]:
cols_num = [ '가입기간', '한달요금', '총지불요금', '가입기간_약정기간_비율', '총지불요금_편차' ]
cols_cat = [ '성별', '고령자여부', '기혼여부', '부양가족여부', '집전화이용여부', '다중회선여부', '인터넷이용방식',
       '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부', '인터넷기술지원서비스사용여부',
       '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법', '스TV_스영화', '부가서비스_5종',
       '전자_가중치','고령자여부_기혼여부','고령자여부_부양가족여부','기혼여부_부양가족여부','집전화이용여부_다중회선여부','집전화이용여부_인터넷이용방식','집전화이용여부_인터넷보안서비스사용여부','집전화이용여부_인터넷백업서비스사용여부','집전화이용여부_기기방화벽서비스사용여부','다중회선여부_인터넷이용방식','다중회선여부_인터넷보안서비스사용여부','다중회선여부_기기방화벽서비스사용여부','인터넷이용방식_인터넷보안서비스사용여부','인터넷이용방식_인터넷백업서비스사용여부','인터넷이용방식_기기방화벽서비스사용여부','인터넷보안서비스사용여부_인터넷백업서비스사용여부','인터넷보안서비스사용여부_기기방화벽서비스사용여부','인터넷백업서비스사용여부_기기방화벽서비스사용여부']
len(cols_num), len(cols_cat)

(5, 36)

In [ ]:
train_ft[cols_cat].nunique() <= 10

성별                           True
고령자여부                        True
기혼여부                         True
부양가족여부                       True
집전화이용여부                      True
다중회선여부                       True
인터넷이용방식                      True
인터넷보안서비스사용여부                 True
인터넷백업서비스사용여부                 True
기기방화벽서비스사용여부                 True
인터넷기술지원서비스사용여부               True
스트리밍TV여부                     True
스트리밍영화여부                     True
약정기간                         True
지로여부                         True
지불방법                         True
스TV_스영화                      True
부가서비스_5종                     True
전자_가중치                       True
고령자여부_기혼여부                   True
고령자여부_부양가족여부                 True
기혼여부_부양가족여부                  True
집전화이용여부_다중회선여부               True
집전화이용여부_인터넷이용방식              True
집전화이용여부_인터넷보안서비스사용여부         True
집전화이용여부_인터넷백업서비스사용여부         True
집전화이용여부_기기방화벽서비스사용여부         True
다중회선여부_인터넷이용방식               True
다중회선여부_인터넷보안서비스사용여부          True
다중회선여부_기기방화벽서비

In [ ]:
mask = train_ft[cols_cat].nunique() <= 10
cols1 = train_ft[cols_cat].nunique().loc[mask].index.tolist()
len(cols1)

36

In [ ]:
enc = ce.count.CountEncoder(return_df=True,handle_unknown=0)

# 학습데이터
tmp = enc.fit_transform(train_ft[cols1]).add_prefix("count_")
train_ft = pd.concat([train_ft,tmp],axis=1)

# 테스트데이터
tmp = enc.transform(test_ft[cols1]).add_prefix("count_")
test_ft = pd.concat([test_ft,tmp],axis=1)

In [ ]:
train_ft.columns

Index(['성별', '고령자여부', '기혼여부', '부양가족여부', '가입기간', '집전화이용여부', '다중회선여부', '인터넷이용방식',
       '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부', '인터넷기술지원서비스사용여부',
       '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법', '한달요금', '총지불요금',
       '가입기간_약정기간_비율', '스TV_스영화', '부가서비스_5종', '총지불요금_편차', '전자_가중치',
       '고령자여부_기혼여부', '고령자여부_부양가족여부', '기혼여부_부양가족여부', '집전화이용여부_다중회선여부',
       '집전화이용여부_인터넷이용방식', '집전화이용여부_인터넷보안서비스사용여부', '집전화이용여부_인터넷백업서비스사용여부',
       '집전화이용여부_기기방화벽서비스사용여부', '다중회선여부_인터넷이용방식', '다중회선여부_인터넷보안서비스사용여부',
       '다중회선여부_인터넷백업서비스사용여부', '다중회선여부_기기방화벽서비스사용여부', '인터넷이용방식_인터넷보안서비스사용여부',
       '인터넷이용방식_인터넷백업서비스사용여부', '인터넷이용방식_기기방화벽서비스사용여부',
       '인터넷보안서비스사용여부_인터넷백업서비스사용여부', '인터넷보안서비스사용여부_기기방화벽서비스사용여부',
       '인터넷백업서비스사용여부_기기방화벽서비스사용여부', 'count_성별', 'count_고령자여부', 'count_기혼여부',
       'count_부양가족여부', 'count_집전화이용여부', 'count_다중회선여부', 'count_인터넷이용방식',
       'count_인터넷보안서비스사용여부', 'count_인터넷백업서비스사용여부', 'count_기기방화벽서비스사용여부',
       'count_인터넷기술지원서비스사용여부', 'count_스트리밍TV여부', 'count_스트리밍영화여부',
    

In [ ]:
train_ft

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷이용방식,인터넷보안서비스사용여부,인터넷백업서비스사용여부,...,count_집전화이용여부_기기방화벽서비스사용여부,count_다중회선여부_인터넷이용방식,count_다중회선여부_인터넷보안서비스사용여부,count_다중회선여부_기기방화벽서비스사용여부,count_인터넷이용방식_인터넷보안서비스사용여부,count_인터넷이용방식_인터넷백업서비스사용여부,count_인터넷이용방식_기기방화벽서비스사용여부,count_인터넷보안서비스사용여부_인터넷백업서비스사용여부,count_인터넷보안서비스사용여부_기기방화벽서비스사용여부,count_인터넷백업서비스사용여부_기기방화벽서비스사용여부
0,1,0,0,0,23,1,0,광,0,1,...,2969,1655,2134,2086,2655,927,2306,894,2581,758
1,1,0,0,0,1,1,0,디지털,0,0,...,2969,1195,2134,2086,851,915,911,2612,2581,2459
2,0,0,0,0,23,1,1,광,0,0,...,1492,1590,1372,949,2655,2318,939,2612,925,774
3,1,0,0,0,11,1,0,디지털,0,0,...,1492,1195,2134,764,851,915,774,2612,925,774
4,1,0,0,0,5,1,1,광,0,0,...,2969,1590,1372,1131,2655,2318,2306,2612,2581,2459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,0,0,0,0,24,1,0,광,0,1,...,1492,1655,2134,764,2655,927,939,894,925,939
4926,0,1,1,0,72,0,0,디지털,1,1,...,221,1195,716,764,834,770,774,803,788,939
4927,0,0,1,1,25,1,0,광,0,0,...,2969,1655,2134,2086,2655,2318,2306,2612,2581,2459
4928,0,0,1,0,48,1,1,광,1,1,...,1492,1590,708,949,590,927,939,803,788,939


In [ ]:
train_ft = train_ft.drop(columns = cols_cat)
test_ft = test_ft.drop(columns = cols_cat)

train_ft.shape , test_ft.shape

((4930, 42), (2113, 42))

In [ ]:
train_ft

,가입기간,한달요금,총지불요금,가입기간_약정기간_비율,총지불요금_편차,다중회선여부_인터넷백업서비스사용여부,count_성별,count_고령자여부,count_기혼여부,count_부양가족여부,...,count_집전화이용여부_기기방화벽서비스사용여부,count_다중회선여부_인터넷이용방식,count_다중회선여부_인터넷보안서비스사용여부,count_다중회선여부_기기방화벽서비스사용여부,count_인터넷이용방식_인터넷보안서비스사용여부,count_인터넷이용방식_인터넷백업서비스사용여부,count_인터넷이용방식_기기방화벽서비스사용여부,count_인터넷보안서비스사용여부_인터넷백업서비스사용여부,count_인터넷보안서비스사용여부_기기방화벽서비스사용여부,count_인터넷백업서비스사용여부_기기방화벽서비스사용여부
0,23,79.10,1783.75,23.0,699.636955,0_1,1,0,0,0,...,2969,1655,2134,2086,2655,927,2306,894,2581,758
1,1,45.40,45.40,1.0,24.435916,0_0,1,0,0,0,...,2969,1195,2134,2086,851,915,911,2612,2581,2459
2,23,104.05,2470.10,23.0,699.636955,1_0,0,0,0,0,...,1492,1590,1372,949,2655,2318,939,2612,925,774
3,11,64.90,697.25,5.5,292.605845,0_0,1,0,0,0,...,1492,1195,2134,764,851,915,774,2612,925,774
4,5,85.20,474.80,5.0,124.615027,1_0,1,0,0,0,...,2969,1590,1372,1131,2655,2318,2306,2612,2581,2459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,24,85.95,2107.15,24.0,650.507418,0_1,0,0,0,0,...,1492,1655,2134,764,2655,927,939,894,925,939
4926,72,63.10,4685.55,24.0,2383.773698,0_1,0,1,1,0,...,221,1195,716,764,834,770,774,803,788,939
4927,25,19.90,505.45,12.5,741.001974,0_0,0,0,1,1,...,2969,1655,2134,2086,2655,2318,2306,2612,2581,2459
4928,48,96.90,4473.45,16.0,1582.439597,1_1,0,0,1,0,...,1492,1590,708,949,590,927,939,803,788,939


In [ ]:
train_ft.columns

Index(['가입기간', '한달요금', '총지불요금', '가입기간_약정기간_비율', '총지불요금_편차',
       '다중회선여부_인터넷백업서비스사용여부', 'count_성별', 'count_고령자여부', 'count_기혼여부',
       'count_부양가족여부', 'count_집전화이용여부', 'count_다중회선여부', 'count_인터넷이용방식',
       'count_인터넷보안서비스사용여부', 'count_인터넷백업서비스사용여부', 'count_기기방화벽서비스사용여부',
       'count_인터넷기술지원서비스사용여부', 'count_스트리밍TV여부', 'count_스트리밍영화여부',
       'count_약정기간', 'count_지로여부', 'count_지불방법', 'count_스TV_스영화',
       'count_부가서비스_5종', 'count_전자_가중치', 'count_고령자여부_기혼여부',
       'count_고령자여부_부양가족여부', 'count_기혼여부_부양가족여부', 'count_집전화이용여부_다중회선여부',
       'count_집전화이용여부_인터넷이용방식', 'count_집전화이용여부_인터넷보안서비스사용여부',
       'count_집전화이용여부_인터넷백업서비스사용여부', 'count_집전화이용여부_기기방화벽서비스사용여부',
       'count_다중회선여부_인터넷이용방식', 'count_다중회선여부_인터넷보안서비스사용여부',
       'count_다중회선여부_기기방화벽서비스사용여부', 'count_인터넷이용방식_인터넷보안서비스사용여부',
       'count_인터넷이용방식_인터넷백업서비스사용여부', 'count_인터넷이용방식_기기방화벽서비스사용여부',
       'count_인터넷보안서비스사용여부_인터넷백업서비스사용여부', 'count_인터넷보안서비스사용여부_기기방화벽서비스사용여부',
       'count_인터넷백업서비스사용여부_기기방화벽서비스사용여부'],
      dtype='

In [ ]:
target = train['이탈여부']
target

0       0
1       1
2       1
3       0
4       1
       ..
4925    0
4926    0
4927    0
4928    0
4929    0
Name: 이탈여부, Length: 4930, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# 학습데이터
train_ft = pd.DataFrame(
    scaler.fit_transform(train_ft),
    columns = train_ft.columns
)

# 테스트 데이터
test_ft = pd.DataFrame(
    scaler.transform(test_ft),
    columns = test_ft.columns
)

# 모델 학습및 교차 검증

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

model = LogisticRegression(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc',n_jobs = -1)
print(scores)
print(np.mean(scores))

[0.84476245 0.83806567 0.84833178 0.84182592 0.8501684 ]
0.844630843088517


In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#확정
from lightgbm import LGBMClassifier

lg_params ={'n_estimators': 155,
 'num_leaves': 4,
 'min_child_samples': 14,
 'learning_rate': 0.0664146308967687,
 'log_max_bin': 6,
 'colsample_bytree': 0.8650285897953071,
 'reg_alpha': 0.09383471573160347,
 'reg_lambda': 0.09005695217909274}

model = LGBMClassifier(**lg_params)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring='roc_auc', n_jobs=-1)
print(scores)
print(np.mean(scores))

[0.84599019 0.84588377 0.85016036 0.84141178 0.85328894]
0.8473470056856705


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et_params = {
    "random_state": 42,
    "n_estimators": 960,
    "max_depth": 8
}


model = ExtraTreesClassifier(**et_params)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring='roc_auc', n_jobs=-1)
print(scores)
print(np.mean(scores))

[0.83855598 0.83432795 0.84628854 0.84279588 0.84742294]
0.8418782576067347


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ab_params = {
    "random_state": 42,
    "n_estimators": 300,
    "learning_rate": 0.06
}

model = AdaBoostClassifier(**ab_params)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring='roc_auc', n_jobs=-1)
print(scores)
print(np.mean(scores))

[0.84743204 0.84638723 0.85027157 0.84396201 0.85187538]
0.847985645441834


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_params ={
    "random_state" : 42,
    "n_estimators": 198,
    "learning_rate": 0.06,
    "max_depth" : 2
}

model = GradientBoostingClassifier(**gb_params)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring='roc_auc', n_jobs=-1)
print(scores)
print(np.mean(scores))

[0.84561617 0.84786861 0.85009052 0.84058895 0.8548071 ]
0.8477942714243907


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier



et_params = {
    "random_state": 42,
    "n_estimators": 221,
    "max_depth": 8
}

ab_params = {
    "random_state": 42,
    "n_estimators": 300,
    "learning_rate": 0.06
}

gb_params ={
    "random_state" : 42,
    "n_estimators": 198,
    "learning_rate": 0.06,
    "max_depth" : 2
}


estimators = [

              ("lr", LogisticRegression(random_state=SEED)),
              ("et", ExtraTreesClassifier(**et_params)),
              ("ab", AdaBoostClassifier(**ab_params)),
              ("gb",GradientBoostingClassifier(**gb_params))

]


params ={
    "estimators" : estimators,
    "voting" : "soft" ,
    "n_jobs" : -1
}

model = VotingClassifier(**params)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc',n_jobs = -1)
print(scores)
np.mean(scores)

[0.84665962 0.84512463 0.85178719 0.84576025 0.85521535]


0.8489094067768391

#모델학습

In [ ]:
model = VotingClassifier(**params)
model.fit(train_ft,target)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('et',
                              ExtraTreesClassifier(max_depth=8,
                                                   n_estimators=221,
                                                   random_state=42)),
                             ('ab',
                              AdaBoostClassifier(learning_rate=0.06,
                                                 n_estimators=300,
                                                 random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(learning_rate=0.06,
                                                         max_depth=2,
                                                         n_estimators=198,
                                                         random_state=42))],
                 n_jobs=-1, voting='soft')

#테스트 데이터 예측

In [ ]:
pred = model.predict_proba(test_ft)[:,1]
pred

array([0.60946194, 0.20200334, 0.48528895, ..., 0.11482707, 0.12870833,
       0.55321999])

# 평가를 위한 제출 파일 생성
- sample_submission.csv 파일을 불러와서 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
2108,test_2108,0.5
2109,test_2109,0.5
2110,test_2110,0.5
2111,test_2111,0.5


In [ ]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.609462
1,test_1,0.202003
2,test_2,0.485289
3,test_3,0.179126
4,test_4,0.449892
...,...,...
2108,test_2108,0.544038
2109,test_2109,0.168645
2110,test_2110,0.114827
2111,test_2111,0.128708


- 예측 결과를 csv 파일로 저장하여 제출

In [ ]:
submit.to_csv(f"{DATA_PATH}50th_try_test_submit.csv",index=False) # 인덱스는 제외하기 위해 False